Traditional classification accuracy cann't be used in masked language model, as there are not one single exact correct awnser when filling a missing text, instead below metrics can be more appropriate in Mask Filling Tasks.

- Perplexity
- Cross Entropy

In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = 'distilbert/distilbert-base-cased'
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [2]:
import torch



def print_mask(model,tokenizer,text,k):
  tokens = tokenizer(text,return_tensors='pt')
  logits = model(**tokens).logits[0]

  mask_token_index = torch.where(tokens['input_ids'] == tokenizer.mask_token_id)[1]
  possible_mask = [tokenizer.decode(id) for id in torch.topk(logits[mask_token_index.clone().detach(),:],k).indices[0]]
  for mask in possible_mask:
    print(f'{text.replace("[MASK]",mask)}')
  print('\n================================================\n')

print_mask(model,tokenizer,'This is a great [MASK].',5)
print_mask(model,tokenizer,'Hello Hamza what is your [MASK].',5)
print_mask(model,tokenizer,'Today was a very busy day, had to do all of the [MASK].',5)
print_mask(model,tokenizer,'Please go to the next room and bring me the [MASK].',5)
print_mask(model,tokenizer,'Please go to the next room and bring some of the [MASK].',5)

This is a great achievement.
This is a great success.
This is a great deal.
This is a great feat.
This is a great celebration.


Hello Hamza what is your name.
Hello Hamza what is your destiny.
Hello Hamza what is your dream.
Hello Hamza what is your motto.
Hello Hamza what is your life.


Today was a very busy day, had to do all of the work.
Today was a very busy day, had to do all of the shopping.
Today was a very busy day, had to do all of the paperwork.
Today was a very busy day, had to do all of the cooking.
Today was a very busy day, had to do all of the talking.


Please go to the next room and bring me the drinks.
Please go to the next room and bring me the coffee.
Please go to the next room and bring me the tray.
Please go to the next room and bring me the keys.
Please go to the next room and bring me the food.


Please go to the next room and bring some of the dishes.
Please go to the next room and bring some of the drinks.
Please go to the next room and bring some of the foo

In [3]:
!pip install datasets==2.16.1

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x

In [4]:
from datasets import load_dataset

data = load_dataset('imdb')
data

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
def tokenize_function(samples):
  result = tokenizer(samples['text'])
  result['word_ids'] = [result.word_ids(i) for i in range(len(result['input_ids']))]
  return result

data_model = data.map(tokenize_function,batched=True,remove_columns=['text','label'])
data_model

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [6]:
chunk_size = 128

examples = data_model['train'][:3]
for i,idx in enumerate(examples['input_ids']):
  print(f'Sample {i} length of {len(idx)}')

def concatinate_examples(examples):
  concatinated_examples = {k:sum(examples[k],[]) for k in examples.keys()}
  return concatinated_examples

def chunk_examples(concatinated_examples):
  chunked_examples = {k:[concatinated_examples[k][i:i+chunk_size] for i in range(0,len(concatinated_examples[k])//chunk_size*chunk_size,chunk_size)] for k in concatinated_examples.keys()}
  return chunked_examples

def concatincate_and_chunk_examples(examples):
  concatinated_examples = concatinate_examples(examples)
  chunked_examples = chunk_examples(concatinated_examples)
  chunked_examples['labels'] = chunked_examples['input_ids'].copy()
  return chunked_examples

len(concatincate_and_chunk_examples(examples)['input_ids'])

Sample 0 length of 391
Sample 1 length of 322
Sample 2 length of 140


6

In [7]:
data_model_ = data_model.map(concatincate_and_chunk_examples,remove_columns=data_model['train'].column_names,batched = True)
data_model_

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 63037
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61623
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 126497
    })
})

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm_probability = 0.15)

training_args = TrainingArguments(
    'Masked-Language-Model-Domain-Adapted',
    evaluation_strategy = 'epoch',
    num_train_epochs = 2,
    per_device_train_batch_size = 20,
    per_device_eval_batch_size = 20,
    push_to_hub = True
)


trainer = Trainer(
    model,
    training_args,
    data_collator = data_collator,
    train_dataset = data_model_['train'],
    eval_dataset = data_model_['test'],
    tokenizer = tokenizer
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.413900,2.290562
2,2.280000,2.221524


TrainOutput(global_step=6304, training_loss=2.3901241902772545, metrics={'train_runtime': 1369.7589, 'train_samples_per_second': 92.041, 'train_steps_per_second': 4.602, 'total_flos': 4177981043718144.0, 'train_loss': 2.3901241902772545, 'epoch': 2.0})

In [14]:
trainer.push_to_hub()

events.out.tfevents.1723208601.63600b14c693.556.1:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HamzaNaser/Masked-Language-Model-Domain-Adapted/commit/e7f34bb7ce5195ab0628588799c9854a0230c3ba', commit_message='End of training', commit_description='', oid='e7f34bb7ce5195ab0628588799c9854a0230c3ba', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
import torch



DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [29]:
device = torch.device('cpu')
model.to(device)

print_mask(model,tokenizer,'This is a great [MASK].',5)
print_mask(model,tokenizer,'Hello Hamza what is your [MASK].',5)
print_mask(model,tokenizer,'Today was a very busy day, had to do all of the [MASK].',5)
print_mask(model,tokenizer,'Please go to the next room and bring me the [MASK].',5)
print_mask(model,tokenizer,'Please go to the next room and bring some of the [MASK].',5)

This is a great movie.
This is a great film.
This is a great show.
This is a great experience.
This is a great classic.


Hello Hamza what is your life.
Hello Hamza what is your name.
Hello Hamza what is your day.
Hello Hamza what is your dream.
Hello Hamza what is your time.


Today was a very busy day, had to do all of the work.
Today was a very busy day, had to do all of the talking.
Today was a very busy day, had to do all of the same.
Today was a very busy day, had to do all of the things.
Today was a very busy day, had to do all of the jobs.


Please go to the next room and bring me the DVD.
Please go to the next room and bring me the present.
Please go to the next room and bring me the box.
Please go to the next room and bring me the rest.
Please go to the next room and bring me the cake.


Please go to the next room and bring some of the rest.
Please go to the next room and bring some of the stuff.
Please go to the next room and bring some of the pictures.
Please go to the next

In [31]:
device = torch.device('cuda')
model.to(device)
trainer.evaluate()

{'eval_loss': 2.2124485969543457,
 'eval_runtime': 165.4376,
 'eval_samples_per_second': 372.485,
 'eval_steps_per_second': 18.629,
 'epoch': 2.0}